In [4]:
# Here, we will be making an attempt to perform transfer learning with the SYSU nighttime dataset
# and we will be doing it in a (potentially naive) binary classifier of Vehicular VS. Non-Vehicular  
# classes. We will only be training some fully connected layers at the end, no conv kernel fine tuning 
#
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 128,128 
model = applications.VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


574717952/574710816 [==============================] - 55s 0us/step


In [5]:
# Let's check out the model's architecture, in order to see what we would like to freeze
# and what we will want to train
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
# Freeze all pre-trained convolutional layers
#    - If pure FC layers stacked on top does not converge, then try unfreezing
#      some of the conv layers 
for layer in model.layers:
    layer.trainable = False
   

# Add two dense layers, a final binary layer with a softmax activation
# Hyperparameters: Number of neurons in both dense layers (X1, X2)
#                  dropout rate (D1)
l = model.output
l = Flatten()(l)
l = Dense(1024, activation='relu')(l)
l = Dropout(0.5)(l)    
l = Dense(1024, activation='relu')(l)
final = Dense(2, activation='softmax')(l)

final_model = Model(inputs=model.input, outputs=final)

# !!! YO, THESE ARE THOUGHTLESSLY CHOSEN HYPERPARAMETERS
#
final_model.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), 
    metrics=["accuracy"])



In [28]:
# Defaults thoughtlessly accepted from here
#
# https://towardsdatascience.com/transfer-learning-using-keras-d804b2e04ef8
#
train_dir = './cropped_split_dataset/train'
validation_dir = './cropped_split_dataset/validation'


# Hyperparameters:
#    - batch_size: thoughtlessly accepted
#    - all data augmentation constants
#
batch_size = 16 
zoom_range = 0.3
width_shift_range = 0.3
height_shift_range=0.3
rotation_range=30


# We will do 5 epochs do investigate the quality of the model, and
# more once our hyperparameters have been selected
epochs = 2 


train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=zoom_range,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    rotation_range=rotation_range)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=zoom_range,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    rotation_range=rotation_range)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    class_mode='categorical'
)


Found 80914 images belonging to 2 classes.
Found 11840 images belonging to 2 classes.


In [29]:
checkpoint = ModelCheckpoint("vgg19_2fc_1024_1024.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [30]:
import os, os.path

num_train_ex = len([file for file in os.listdir(train_dir + '/cars/')]) + len([file for file in os.listdir(train_dir + '/non-cars/')])
num_validation_ex = len([file for file in os.listdir(validation_dir + '/cars/')]) + len([file for file in os.listdir(validation_dir + '/non-cars/')])

print(num_train_ex)
print(num_validation_ex)

80914
11840


In [31]:
final_model.fit_generator(
    train_generator,
    steps_per_epoch=int(num_train_ex/batch_size),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=int(num_validation_ex/batch_size), 
    use_multiprocessing=True, 
    callbacks=[checkpoint, early])



Epoch 1/2
5057/5057 [==============================] - 15333s 3s/step - loss: 0.3160 - acc: 0.8588 - val_loss: 0.2645 - val_acc: 0.8861

Epoch 00001: val_acc improved from -inf to 0.88615, saving model to vgg19_2fc_1024_1024.h5
Epoch 2/2
5057/5057 [==============================] - 15325s 3s/step - loss: 0.2977 - acc: 0.8689 - val_loss: 0.2531 - val_acc: 0.8922

Epoch 00002: val_acc improved from 0.88615 to 0.89223, saving model to vgg19_2fc_1024_1024.h5
